<a href="https://colab.research.google.com/github/valterlucena/recuperacao-informacao/blob/master/query-expansion/query_expansion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

import collections
import matplotlib.pyplot as plt
%matplotlib inline

import heapq as hp

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Introdução

Nesta atividade exercitaremos conceitos sobre expansão de consultas.  Os dados utilizados são de textos de notícias resultados da utilização de técnicas de crawling e scrapping. Os scripts para a coleta desses dados podem ser encontrados [neste](https://github.com/valterlucena/ri_lab_01) repositório.

Primeiramente, vamos importar os dados que serão utilizados.


In [0]:
DATA_URL = 'https://raw.githubusercontent.com/valterlucena/ri_lab_01/master/output/results.csv'
news = pd.read_csv(DATA_URL).replace(np.nan, '', regex=True)

Agora iremos criar nosso índice invertido, para ser utilizado mais adiante. Utilizaremos a função tokenize da biblioteca NLTK associada à uma expressão regular, que considerará como token apenas as sequências de caracteres não-especiais (com exceção do hífen) ou numéricos que não formem stopwords e quem possuam mais que 2 caracteres.

In [0]:
toker = RegexpTokenizer('''\w+[-']*\w*''')
stop_words = stopwords.words('portuguese')

def isValid(token):
  return token not in stop_words and not bool(re.search(r'\d', token)) and len(token) > 2

def build_index(documents):
  index = {}
  n = 0
  for document in documents:
    n += 1
    tokens = [token for token in toker.tokenize(document.lower()) if isValid(token)]
    for token in tokens:
      occurrence = tokens.count(token)
      if token not in index:
        index[token] = {}
      if n not in index[token]:
        index[token][n] = occurrence
  return index

index = build_index(news.text)

# Medidas de associação.

O objetivo das técnicas de expansão de consultas é melhorar a efetividade de seus resultados a partir do casamento de termos que são relacionados entre si.
A chave para esta efetividade é escolher palavras que são apropriadas ao contexto da consulta. 

As medidas de associação são uma parte importante de muitas técnicas de expansão de consulta. Utilizaremos algumas dessas medidas para rankear os termos do índice em relação às consultas de um termo realizadas na [atividade anterior](https://github.com/valterlucena/recuperacao-informacao/blob/master/index-query/index_query.ipynb), com o objetivo de selecionar os melhores termos para expandir essas consultas.

Para isso, iremos precisar de algumas funções auxiliares.

In [0]:
'''
  Calcula o total de documentos que contém um determinado termo
''' 
def count_docs(termo, index):
  return len(index[termo].keys())

'''
  Calcula o total de documentos que contém os termos a e b
'''
def count_docs_conj(a, b, index):
  docs_a = index[a].keys()
  docs_b = index[b].keys()
  count = 0
  for doc in docs_a:
    if doc in docs_b:
      count += 1
  return count  

'''
  Retorna o total de documentos na base de dados.
'''
def total_docs(index):
  docs = []
  for _,inv_lists in index.items():
    for doc,_ in inv_lists.items():
      if doc not in docs:
        docs.append(doc)
  return len(docs)

Essas funções auxiliarão no cálculo das medidas de associação que utilizaremos, que são:

* Mutual Information Measure (MIM)
* Expected Mutual Information Measure (EMIM)
* Pearson's Chi-squared (X²)
* Dice's coefficient (Dice)

Abaixo, criaremos funções que calculam essas medidas.

In [0]:
'''
  Medidas de associação de termos
'''

'''
  Mutal Information Measure
'''
def mutual_information(a, b, index):
  n_a = count_docs(a, index)
  n_b = count_docs(b, index)
  n_ab = count_docs_conj(a, b, index)
  mim = n_ab / (n_a * n_b)
  return mim

'''
  Expected Mutual Information Measure
'''
def expected_mutual_information(a, b, index, n):
  n_a = count_docs(a, index)
  n_b = count_docs(b, index)
  n_ab = count_docs_conj(a, b, index)
  exp = n * (n_ab / (n_a * n_b))
  emim = n_ab * np.log(exp) if exp != 0 else 0
  return emim

'''
  Pearson's Chi-squared
'''
def chi_square(a, b, index, n):
  n_a = count_docs(a, index)
  n_b = count_docs(b, index)
  n_ab = count_docs_conj(a, b, index)
  chi = (n_ab - (1 / n) * n_a * n_b) ** 2 / (n_a * n_b)
  return chi

'''
  Dice's coefficient
'''
def dice_coefficient(a, b, index):
  n_a = count_docs(a, index)
  n_b = count_docs(b, index)
  n_ab = count_docs_conj(a, b, index)
  dice = n_ab / (n_a + n_b)
  return dice

Agora, vamos criar tabelas que mostram os 10 termos mais relevantes do índice de acordo com cada um dos termos da consulta.

In [0]:
'''
  Cria uma lista de tuplas contendo o valor de uma determinada métrica para um 
  posting
'''
def build_metric_list(termo, index, metric):
  pairs = []
  n = total_docs(index)
  for posting in index.keys():
    if posting != termo:
      if metric == 'mim':
        metric_value = mutual_information(termo, posting, index)
      elif metric == 'emim':
        metric_value = expected_mutual_information(termo, posting, index, n)
      elif metric == 'chi-square':
        metric_value = chi_square(termo, posting, index, n)
      elif metric == 'dice':
        metric_value = dice_coefficient(termo, posting, index)
      pairs.append((posting, metric_value))      
  pairs = sorted(pairs, key = lambda x: x[1], reverse=True)
  return pairs
  
def build_term_metrics(termo, index):
  metricas = {}
  metricas['mim'] = build_metric_list(termo, index, 'mim')
  metricas['emim'] = build_metric_list(termo, index, 'emim')
  metricas['chi-square'] = build_metric_list(termo, index, 'chi-square')
  metricas['dice'] = build_metric_list(termo, index, 'dice')
  return metricas

'''
  Constrói um dataframe com o top 10 de palavras mais associadas à um termo de
  acordo com cada uma das métricas
'''
def build_table(termo, index):
  metricas = build_term_metrics(termo, index)
  mim = [k for k,_ in metricas['mim'][0:10]]
  emim = [k for k,_ in metricas['emim'][0:10]]
  chi_square = [k for k,_ in metricas['chi-square'][0:10]]
  dice = [k for k,_ in metricas['dice'][0:10]]
  data = {'MIM': mim, 'EMIM': emim, 'X²': chi_square, 'Dice': dice}
  return data

Os termos a serem consultados são:

In [0]:
termos = ['presidente', 'lula', 'bolsonaro', 'governo', 'federal']

Tabela para a palavra **presidente**:

In [0]:
pd.DataFrame(build_table(termos[0], index))

,MIM,EMIM,X²,Dice
0,requereu,bolsonaro,jair,bolsonaro
1,penal,jair,candidato,brasil
2,especial,brasil,processo,jair
3,embu,direitos,bolsonaro,ser
4,artes,ser,gente,governo
5,nando,governo,direitos,país
6,moura,apenas,corrupção,direitos
7,injúria,processo,quer,contra
8,difamação,contra,apenas,dia
9,requerida,fazer,pois,hoje


Tabela para a palavra **lula**:

In [0]:
pd.DataFrame(build_table(termos[1], index))

,MIM,EMIM,X²,Dice
0,elio,ex-presidente,luiz,ex-presidente
1,gaspari,dizer,ex-presidente,dizer
2,resolve,democracia,dizer,democracia
3,pedirá,luiz,inocência,luiz
4,inocência,prisão,inácio,legislação
5,negocio,legislação,legislação,ato
6,trocaria,silva,curitiba,silva
7,relativo,dentro,visão,dentro
8,conforto,ato,condenou,prisão
9,assemelhariam,direitos,democracia,curitiba


Tabela para a palavra **bolsonaro**:

In [0]:
pd.DataFrame(build_table(termos[2], index))

,MIM,EMIM,X²,Dice
0,requereu,jair,jair,jair
1,especial,presidente,onde,presidente
2,embu,onde,presidente,ser
3,artes,governo,deputados,governo
4,nando,ser,vai,brasil
5,moura,brasil,antes,país
6,injúria,país,político,onde
7,difamação,vai,final,sobre
8,requerida,antes,semana,vai
9,promotoria,política,governo,ter


Tabela para a palavra **governo**:

In [0]:
pd.DataFrame(build_table(termos[3], index))

,MIM,EMIM,X²,Dice
0,requereu,reforma,reforma,bolsonaro
1,penal,bolsonaro,previdência,ser
2,corre,brasil,outra,brasil
3,especial,ser,tudo,presidente
4,embu,política,falta,todos
5,artes,justiça,justiça,política
6,nando,todos,cerca,país
7,moura,federal,deputados,contra
8,injúria,outra,política,federal
9,difamação,tudo,comissão,ter


Tabela para a palavra **federal**:

In [0]:
pd.DataFrame(build_table(termos[4], index)) 

,MIM,EMIM,X²,Dice
0,defendia,ministro,ministro,ministro
1,lobista,justiça,justiça,justiça
2,denunciou,direitos,tribunal,direitos
3,furnas,prisão,prisão,governo
4,dino,governo,fernando,prisão
5,miraglia,tribunal,investigações,ano
6,comeu,polícia,humanos,nacional
7,pão,desde,polícia,ser
8,amaçou,deputado,direitos,ter
9,helicóptero,processo,civil,público


Podemos notar que o coeficiente de **Dice**, **EMIM** e **Chi-squared** retornam resultados bem relacionados ao termo de pesquisa, já que, ao contrário de **MIM**, não priorizam sempre palavras de baixa ocorrência, fazendo com que os resultados sejam mais generalistas. No entanto, os melhores resultados foram o do coeficiente de **Dice**, já que ele retorna resultados parecidos com as outras métricas generalistas com um cálculo bem mais simples. 

# Expansão de consulta.

Utilizando o coeficiente de **Dice**, vamos utilizar a técnica termo-por-vez expandido-a com os top-3, top-5 e top-10 documentos. 


In [0]:
def get_top_results(k, results):
  top = []
  for i in range(1, k+1):
    if results:
      current_top = hp._heappop_max(results)
      top.append(current_top)
  return top

def term_at_a_time_retrieval(query, index, k):
  accumulators = {}
  inverted_lists = []
  for term in query.split():
    inverted_list = index[term]
    inverted_lists.append(inverted_list)
  for inv_list in inverted_lists:
    for doc,occurrence in inv_list.items():
      if doc in accumulators:
        accumulators[doc] += occurrence
      else:
        accumulators[doc] = occurrence
  results = [(score, document) for document,score in accumulators.items()]
  hp._heapify_max(results)
  top = get_top_results(k, results)
  return top

def index_top_docs(query, k):
  top_docs = [doc for score, doc in term_at_a_time_retrieval(query, index, k)]
  top_docs_rank = {}
  for posting in index:
    for document,score in index[posting].items():
      if document in top_docs:
        if posting in top_docs_rank:
          top_docs_rank[posting][document] = score
        else:
          top_docs_rank[posting] = {}
          top_docs_rank[posting][document] = score
  return top_docs_rank

In [0]:
def expand_query(term, k):
  new_index = index_top_docs(term, k)
  return build_table(term,new_index)['Dice']

In [0]:
def build_expand_table(term):
  top_3 = expand_query(term, 3)
  top_5 = expand_query(term, 5)
  top_10 = expand_query(term, 10)
  return {'Top 3': top_3, 'Top 5': top_5, 'Top 10': top_10}

Para **presidente**, temos:

In [0]:
pd.DataFrame(build_expand_table(termos[0]))

,Top 3,Top 5,Top 10
0,estado,governo,bolsonaro
1,vezes,jair,governo
2,janeiro,bolsonaro,jair
3,governo,contra,contra
4,jair,grupo,brasil
5,bolsonaro,ministro,poder
6,onde,direitos,onde
7,parte,ainda,ser
8,contra,janeiro,fazer
9,havia,durante,país


Para **lula**, temos:

In [0]:
pd.DataFrame(build_expand_table(termos[1]))

,Top 3,Top 5,Top 10
0,justiça,brasil,brasil
1,ano,justiça,bolsonaro
2,outros,pessoas,disse
3,ser,ano,ser
4,dia,porque,contra
5,brasil,outros,todos
6,política,ser,direitos
7,liberdade,contra,democracia
8,federal,dia,governo
9,preso,política,jair


Para **bolsonaro**, temos:

In [0]:
pd.DataFrame(build_expand_table(termos[2]))

,Top 3,Top 5,Top 10
0,governo,governo,governo
1,jair,jair,jair
2,tudo,presidente,ser
3,passado,ainda,presidente
4,presidente,anos,ainda
5,ter,tempo,brasil
6,ainda,onde,ter
7,poder,ser,contra
8,institucional,contra,país
9,executivo,país,política


Para **governo**, temos:

In [0]:
pd.DataFrame(build_expand_table(termos[3]))

,Top 3,Top 5,Top 10
0,janeiro,jair,bolsonaro
1,jair,bolsonaro,ser
2,bolsonaro,durante,jair
3,durante,contra,política
4,ser,presidente,sobre
5,crime,política,federal
6,contra,grupo,reforma
7,brasil,direitos,ainda
8,presidente,ainda,público
9,política,anos,anos


Para **federal**, temos:

In [0]:
pd.DataFrame(build_expand_table(termos[4]))

,Top 3,Top 5,Top 10
0,estado,brasil,governo
1,direito,ministério,direitos
2,havia,público,público
3,brasil,processo,presidente
4,assim,janeiro,ministério
5,ter,governo,jair
6,presidência,ser,bolsonaro
7,pouco,contra,ser
8,ministério,havia,após
9,público,após,ministro
